In [1]:
import requests
import pandas as pd
from apscheduler.schedulers.background import BackgroundScheduler
import time
import os

url = "https://recoglass.net/api/get.php?key=SPXEGN1TGSB2IKT9&merge=10min&ls=0&le=5000&ext=json"
max_entries = 10000  

def fetch_data():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)

        
        df.drop_duplicates(inplace=True)

        
        if os.path.exists('real_time_co2_data.csv'):
            existing_df = pd.read_csv('real_time_co2_data.csv')

            
            last_timestamp = existing_df['datetime'].max() if 'datetime' in existing_df.columns else None

            
            if last_timestamp:
                df = df[df['datetime'] > last_timestamp]

            
            new_df = pd.concat([df, existing_df])

            
            if len(new_df) > max_entries:
                new_df = new_df.tail(max_entries)

            
            new_df.to_csv('real_time_co2_data.csv', index=False)
        else:
           
            df.to_csv('real_time_co2_data.csv', index=False)

        print(f"Data fetched and saved at {time.ctime()}")
    except Exception as e:
        print(f"Error fetching data: {e}")

scheduler = BackgroundScheduler()
scheduler.add_job(fetch_data, 'interval', minutes=10)
scheduler.start()

try:
    while True:
        time.sleep(2)
except (KeyboardInterrupt, SystemExit):
    scheduler.shutdown()


Data fetched and saved at Mon Dec 16 10:28:19 2024
